In [ ]:
!pip install wandb

In [2]:
cd /content/drive/MyDrive/DeepLearningTasks/FaceRecognition-ExpertMode

/content/drive/MyDrive/DeepLearningTasks/FaceRecognition-ExpertMode


### Settig Parameters and Imports

In [35]:
%%writefile config.py

width = height = 112
batch_size = 16
epochs = 30

Overwriting config.py


In [45]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras import Input
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tqdm import tqdm
from config import *

### Preparing Dataset

In [4]:
data_path = '/content/drive/MyDrive/7-7 dataset'

data_generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)

train_data = data_generator.flow_from_directory(
    data_path,
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)


val_data = data_generator.flow_from_directory(
    data_path,
    target_size=(width, height),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


label_map = (train_data.class_indices)
np.save('label_map.npy', label_map) 
num_classes = len(np.bincount(train_data.labels))

print('num_classes: {}'.format(num_classes))
print(np.bincount(train_data.labels))
print(np.bincount(val_data.labels))

Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.
num_classes: 14
[73 81 82 80 84 71 80 60 80 77 78 90 74 81]
[18 20 20 20 21 17 19 15 20 19 19 22 18 20]


### Define Model and wandb Initializing

1-1- My Model

In [46]:
class MyModel(tf.keras.Model):
  def __init__(self, num_class, input_shape):
    super().__init__()
    self.conv1 = Conv2D(32, (3, 3), activation='relu', input_shape = input_shape)
    self.conv2 = Conv2D(64, (3, 3), activation='relu')
    self.conv3 = Conv2D(128, (3, 3), activation='relu')
    self.conv4 = Conv2D(256, (3, 3), activation='relu')
    self.maxpool = MaxPooling2D()
    self.flatten = Flatten()
    self.dropout = Dropout(0.2)
    self.fc1 = Dense(128, activation='relu')
    self.fc2 = Dense(64, activation='relu')
    self.fc3= Dense(num_class, activation='softmax')
    self.dim = input_shape


  def call(self, x):
    conv1 = self.conv1(x)
    maxpool1 = self.maxpool(conv1)

    conv2 = self.conv2(maxpool1)
    maxpool2 = self.maxpool(conv2)

    conv3 = self.conv3(maxpool2)
    maxpool3 = self.maxpool(conv3)

    conv4 = self.conv4(maxpool3)
    maxpool4 = self.maxpool(conv4)

    flatten = self.flatten(maxpool4)
    dropout = self.dropout(flatten)
    
    fc1 = self.fc1(dropout)
    dropout = self.dropout(fc1)
    
    fc2 = self.fc2(dropout)
    dropout = self.dropout(fc2)
    
    output = self.fc3(dropout)

    return output

  def build_graph(self):
      x = Input(shape=(self.dim))
      return tf.keras.Model(inputs=[x], outputs=self.call(x))


input_shape = (width ,height ,3)
model = MyModel(num_classes, (input_shape))
model.build((None, *input_shape))
model.build_graph().summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 110, 110, 32) 896         input_6[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_8 (MaxPooling2D)  multiple             0           conv2d_40[0][0]                  
                                                                 conv2d_41[0][0]                  
                                                                 conv2d_42[0][0]                  
                                                                 conv2d_43[0][0]            

In [ ]:
wandb.init(project='FaceRecognition-ExpertMode')
config = wandb.config
config.learning_rate=0.0001

### Define Loss and Training Loop

In [48]:
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

In [49]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_acc = tf.keras.metrics.CategoricalAccuracy(name='train_acc')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_acc = tf.keras.metrics.CategoricalAccuracy(name='val_acc')

In [50]:
def train_step(images, y):
  with tf.GradientTape() as tape:
    y_pred =  model(images, training = True)
    loss = loss_function(y, y_pred)

  # calculate gradient
  gradient = tape.gradient(loss, model.trainable_variables)
  
  # update
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))

  train_loss(loss)
  train_acc(y, y_pred)

In [51]:
def val_step(images, y):
  y_pred = model(images, training=False)
  loss = loss_function(y, y_pred)

  val_loss(loss)
  val_acc(y, y_pred)

In [52]:
def train():
  
  train_step_per_epoch = train_data.samples // batch_size
  val_step_per_epoch = val_data.samples // batch_size

  for epoch in range(epochs):
    train_loss.reset_state()
    train_acc.reset_state()
    val_loss.reset_state()
    val_acc.reset_state()

    
    for _ in tqdm(range(train_step_per_epoch)):
      images, labels = next(train_data)
      train_step(images, labels)
    
    for _ in tqdm(range(val_step_per_epoch)):
      images, labels = next(val_data)
      val_step(images, labels)

    print('epoch: {}'.format(epoch + 1))
    print('loss: {}'.format(train_loss.result()))
    print('accuracy: {}'.format(train_acc.result()))
    print('val_loss: {}'.format(val_loss.result()))
    print('val_accuracy: {}'.format(val_acc.result()))


    # log metrics using wandb.log
    wandb.log({'epochs':  epoch + 1,
                'loss': np.mean(train_loss.result()),
                'acc': float(train_acc.result()), 
                'val_loss': np.mean(val_loss.result()),
                'val_acc':float(val_acc.result())})

In [53]:
train()

100%|██████████| 16/16 [00:04<00:00,  3.83it/s]


epoch: 1
loss: 2.636568069458008
accuracy: 0.07534883916378021
val_loss: 2.611361503601074
val_accuracy: 0.0833333358168602


100%|██████████| 16/16 [00:04<00:00,  3.90it/s]


epoch: 2
loss: 2.592245101928711
accuracy: 0.13209302723407745
val_loss: 2.482846975326538
val_accuracy: 0.2976190447807312


100%|██████████| 16/16 [00:04<00:00,  3.90it/s]


...

epoch: 48
loss: 0.07747302204370499
accuracy: 0.968372106552124
val_loss: 0.8426114916801453
val_accuracy: 0.85317462682724


100%|██████████| 16/16 [00:03<00:00,  4.05it/s]


epoch: 49
loss: 0.0809091255068779
accuracy: 0.9770220518112183
val_loss: 0.8185986876487732
val_accuracy: 0.8492063283920288


100%|██████████| 16/16 [00:03<00:00,  4.03it/s]

epoch: 50
loss: 0.06746424734592438
accuracy: 0.9720930457115173
val_loss: 0.9579612612724304
val_accuracy: 0.8134920597076416


In [24]:
model.save_weights(filepath='checkpoint',save_format='HDF5')